In [1]:
from modules.statistical import *
import copy

modules.preprocessing successfull loaded: 100%|███| 6/6 [00:03<00:00,  1.73it/s]


In [2]:
g = read_graph("/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs7/GERMAN/withClass/0.1/qualitative/mlna_1/graph_storage/Attribute_1_mln_800_4_2024_05_11_03_26_31.gml.gz")


In [25]:
nx.is_bipartite(g)

True

In [3]:
def plotG(g,withLabel=False):
    colors = nx.get_edge_attributes(g, 'color').values()
    colorsN = nx.get_node_attributes(g, 'color').values()
    nx.draw(
        g,
        edge_color=colors,
        node_color=colorsN,
        with_labels=withLabel)
    plt.show()
    # plt.savefig("graphe.png", dpi=700)
    plt.close()

In [4]:
p,x_train,x_test,y_train,y_test = read_model('/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs7/GERMAN/mlna_preprocessing/model_storage/preprocessing_2024_05_11_03_26_31.sav')

In [5]:
x_train.shape

(800, 61)

In [6]:
x_test.shape

(200, 61)

In [7]:
PERSONS = get_persons(x_train)
PERSONS_T = get_persons(x_test)
target_variable = 'Class'

In [8]:
extracts_g = {
    "Att_DEGREE_GLO" : [],
    "Att_INTRA_GLO" : [],
    "Att_INTER_GLO" : [],
    "Att_COMBINE_GLO" : [],
    "Att_M_INTRA_GLO" : [],
    "Att_M_INTER_GLO" : [],
    "Att_M_COMBINE_GLO" : []
}
extracts_p = {
    "Att_DEGREE_PER" : [], # GLO
    "Att_COMBINE_PER" : [],
    "YN_COMBINE_PER" : [],
    "YP_COMBINE_PER" : [],# COM
    "Att_INTER_PER" : [],
    "YN_INTER_PER" : [],
    "YP_INTER_PER" : [],#INTER
    "Att_INTRA_PER" : [],
    "YN_INTRA_PER" : [],
    "YP_INTRA_PER" : [],#INTRA
    "Att_M_COMBINE_PER" : [],
    "Att_M_INTER_PER" : [],
    "Att_M_INTRA_PER" : []
}
extracts_g_t = copy.deepcopy(extracts_g)
extracts_p_t = copy.deepcopy(extracts_p)

In [9]:
graphWithClass= True
alpha=0.1

In [10]:
copT = x_train.copy(deep=True)
copT[target_variable] = y_train.copy(deep=True)

In [11]:
for borrower in PERSONS:
    # compute descriptors for current borrower
    current = extract_descriptors_from_graph_model(
        graph=g,
        y_graph=removeEdge(g, 1, copT.loc[borrower,target_variable], borrower),
        graphWithClass=graphWithClass,
        alpha=alpha,
        borrower=borrower,
        layers=1
    )
    # extract descriptors context
    # print(f"{list(current.keys())} <--> {list(extracts_g.keys())} <--> {list(extracts_p.keys())}")
    for key in list(current.keys()):
        if 'GLO' in key:
            extracts_g[key].append(current[key])
        else:
            extracts_p[key].append(current[key])
    current = None

In [12]:
for borrower in PERSONS_T:
    # compute descriptors for current borrower
    grf = add_specific_loan_in_mlg(g, x_test.loc[[borrower]],[p[8][0]])
    current = extract_descriptors_from_graph_model(
        graph=grf,
        y_graph=grf,
        graphWithClass=graphWithClass,
        alpha=alpha,
        borrower=borrower,
        layers=1
    )
    # extract descriptors context
    for key in list(current.keys()):
        if 'GLO' in key:
            extracts_g_t[key].append(current[key])
        else:
            extracts_p_t[key].append(current[key])
    current = None

In [13]:
pd.DataFrame(extracts_g).describe()

,Att_DEGREE_GLO,Att_INTRA_GLO,Att_INTER_GLO,Att_COMBINE_GLO,Att_M_INTRA_GLO,Att_M_INTER_GLO,Att_M_COMBINE_GLO
count,800.000000,800.000000,800.000000,800.000000,800.000000,8.000000e+02,8.000000e+02
mean,244.067500,0.150777,0.007294,0.007831,0.000078,5.952353e-04,5.933022e-04
std,65.217019,0.000006,0.001941,0.001934,0.000061,1.886138e-09,2.283522e-07
min,47.000000,0.150755,0.001429,0.001987,0.000050,5.952344e-04,5.931984e-04
25%,212.000000,0.150776,0.006340,0.006880,0.000050,5.952344e-04,5.931984e-04
50%,229.000000,0.150779,0.006846,0.007384,0.000067,5.952349e-04,5.932612e-04
75%,308.000000,0.150780,0.009197,0.009727,0.000072,5.952351e-04,5.932809e-04
max,308.000000,0.150780,0.009197,0.009727,0.000318,5.952426e-04,5.941996e-04


In [14]:
pd.DataFrame(extracts_g_t).describe()

,Att_DEGREE_GLO,Att_INTRA_GLO,Att_INTER_GLO,Att_COMBINE_GLO,Att_M_INTRA_GLO,Att_M_INTER_GLO,Att_M_COMBINE_GLO
count,200.000000,200.000000,200.000000,200.000000,200.000000,2.000000e+02,2.000000e+02
mean,245.165000,0.150781,0.007349,0.007885,0.000078,5.651303e-04,5.633100e-04
std,70.800646,0.000003,0.002106,0.002098,0.000066,1.307885e-08,2.593465e-07
min,48.000000,0.150770,0.001485,0.002042,0.000049,5.651245e-04,5.631958e-04
25%,213.000000,0.150779,0.006393,0.006932,0.000049,5.651245e-04,5.631958e-04
50%,230.000000,0.150782,0.006898,0.007436,0.000065,5.651278e-04,5.632608e-04
75%,309.000000,0.150782,0.009248,0.009777,0.000070,5.651288e-04,5.632811e-04
max,309.000000,0.150784,0.009248,0.009777,0.000308,5.651756e-04,5.642090e-04


In [15]:
pd.DataFrame(extracts_p).describe()

,Att_DEGREE_PER,Att_COMBINE_PER,YN_COMBINE_PER,YP_COMBINE_PER,Att_INTER_PER,YN_INTER_PER,YP_INTER_PER,Att_INTRA_PER,YN_INTRA_PER,YP_INTRA_PER,Att_M_COMBINE_PER,Att_M_INTER_PER,Att_M_INTRA_PER
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,8.000000e+02,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,244.067500,0.315947,0.000063,0.000027,0.473693,0.000011,4.455385e-06,0.000453,0.000169,0.000071,0.317367,0.023799,0.904505
std,65.217019,0.000129,0.000013,0.000013,0.000010,0.000001,6.788766e-07,0.000367,0.000038,0.000038,0.000001,0.000001,0.000001
min,47.000000,0.315891,0.000048,0.000012,0.473689,0.000005,1.677072e-06,0.000293,0.000122,0.000026,0.317363,0.023795,0.904501
25%,212.000000,0.315891,0.000048,0.000012,0.473689,0.000011,4.607590e-06,0.000293,0.000123,0.000027,0.317367,0.023798,0.904504
50%,229.000000,0.315926,0.000058,0.000032,0.473692,0.000011,4.634963e-06,0.000394,0.000151,0.000088,0.317367,0.023798,0.904504
75%,308.000000,0.315937,0.000078,0.000042,0.473693,0.000011,4.634963e-06,0.000426,0.000213,0.000116,0.317369,0.023800,0.904506
max,308.000000,0.316450,0.000078,0.000042,0.473731,0.000011,4.634963e-06,0.001889,0.000213,0.000117,0.317369,0.023800,0.904506


In [16]:
pd.DataFrame(extracts_p_t).describe()

,Att_DEGREE_PER,Att_COMBINE_PER,YN_COMBINE_PER,YP_COMBINE_PER,Att_INTER_PER,YN_INTER_PER,YP_INTER_PER,Att_INTRA_PER,YN_INTRA_PER,YP_INTRA_PER,Att_M_COMBINE_PER,Att_M_INTER_PER,Att_M_INTRA_PER
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,2.000000e+02,200.000000,200.000000,200.000000,200.000000,2.000000e+02,200.000000
mean,245.165000,0.450257,0.000092,0.000035,0.900047,0.000012,4.703687e-06,0.000467,0.000173,0.000066,0.497493,9.045706e-02,0.904528
std,70.800646,0.000219,0.000019,0.000018,0.000040,0.000002,2.445146e-07,0.000398,0.000037,0.000037,0.000010,8.422079e-07,0.000019
min,48.000000,0.450160,0.000067,0.000016,0.900029,0.000011,4.632856e-06,0.000292,0.000123,0.000027,0.497489,9.045628e-02,0.904521
25%,213.000000,0.450160,0.000081,0.000016,0.900029,0.000011,4.632856e-06,0.000292,0.000151,0.000027,0.497489,9.045628e-02,0.904521
50%,230.000000,0.450215,0.000088,0.000038,0.900039,0.000011,4.634986e-06,0.000392,0.000166,0.000072,0.497490,9.045697e-02,0.904524
75%,309.000000,0.450232,0.000112,0.000047,0.900042,0.000011,4.635951e-06,0.000423,0.000213,0.000089,0.497491,9.045749e-02,0.904525
max,309.000000,0.451015,0.000112,0.000061,0.900185,0.000018,5.560233e-06,0.001846,0.000213,0.000117,0.497526,9.045929e-02,0.904593


# old config

In [17]:
grf1 = add_specific_loan_in_mlg(g, x_test,[p[8][0]])

In [18]:
extracts_g1 = {
    "Att_DEGREE_GLO" : [],
    "Att_INTRA_GLO" : [],
    "Att_INTER_GLO" : [],
    "Att_COMBINE_GLO" : [],
    "Att_M_INTRA_GLO" : [],
    "Att_M_INTER_GLO" : [],
    "Att_M_COMBINE_GLO" : []
}
extracts_p1 = {
    "Att_DEGREE_PER" : [], # GLO
    "Att_COMBINE_PER" : [],
    "YN_COMBINE_PER" : [],
    "YP_COMBINE_PER" : [],# COM
    "Att_INTER_PER" : [],
    "YN_INTER_PER" : [],
    "YP_INTER_PER" : [],#INTER
    "Att_INTRA_PER" : [],
    "YN_INTRA_PER" : [],
    "YP_INTRA_PER" : [],#INTRA
    "Att_M_COMBINE_PER" : [],
    "Att_M_INTER_PER" : [],
    "Att_M_INTRA_PER" : []
}
extracts_g_t1 = copy.deepcopy(extracts_g1)
extracts_p_t1 = copy.deepcopy(extracts_p1)

In [19]:
for borrower in PERSONS:
    # compute descriptors for current borrower
    current = extract_descriptors_from_graph_model(
        graph=g,
        y_graph=removeEdge(grf1, 1, copT.loc[borrower,target_variable], borrower),
        graphWithClass=graphWithClass,
        alpha=alpha,
        borrower=borrower,
        layers=1
    )
    # extract descriptors context
    # print(f"{list(current.keys())} <--> {list(extracts_g.keys())} <--> {list(extracts_p.keys())}")
    for key in list(current.keys()):
        if 'GLO' in key:
            extracts_g1[key].append(current[key])
        else:
            extracts_p1[key].append(current[key])
    current = None

In [20]:
for borrower in PERSONS_T:
    # compute descriptors for current borrower
    current = extract_descriptors_from_graph_model(
        graph=grf1,
        y_graph=grf1,
        graphWithClass=graphWithClass,
        alpha=alpha,
        borrower=borrower,
        layers=1
    )
    # extract descriptors context
    for key in list(current.keys()):
        if 'GLO' in key:
            extracts_g_t1[key].append(current[key])
        else:
            extracts_p_t1[key].append(current[key])
    current = None

In [21]:
pd.DataFrame(extracts_g1).describe()

,Att_DEGREE_GLO,Att_INTRA_GLO,Att_INTER_GLO,Att_COMBINE_GLO,Att_M_INTRA_GLO,Att_M_INTER_GLO,Att_M_COMBINE_GLO
count,800.000000,800.000000,800.000000,800.000000,800.000000,8.000000e+02,8.000000e+02
mean,244.067500,0.150777,0.007294,0.007831,0.000078,5.952353e-04,5.933022e-04
std,65.217019,0.000006,0.001941,0.001934,0.000061,1.886138e-09,2.283522e-07
min,47.000000,0.150755,0.001429,0.001987,0.000050,5.952344e-04,5.931984e-04
25%,212.000000,0.150776,0.006340,0.006880,0.000050,5.952344e-04,5.931984e-04
50%,229.000000,0.150779,0.006846,0.007384,0.000067,5.952349e-04,5.932612e-04
75%,308.000000,0.150780,0.009197,0.009727,0.000072,5.952351e-04,5.932809e-04
max,308.000000,0.150780,0.009197,0.009727,0.000318,5.952426e-04,5.941996e-04


In [22]:
pd.DataFrame(extracts_g_t1).describe()

,Att_DEGREE_GLO,Att_INTRA_GLO,Att_INTER_GLO,Att_COMBINE_GLO,Att_M_INTRA_GLO,Att_M_INTER_GLO,Att_M_COMBINE_GLO
count,200.000000,200.000000,200.000000,200.000000,200.000000,2.000000e+02,2.000000e+02
mean,306.775000,0.150929,0.009645,0.010114,0.000062,5.031196e-04,5.016534e-04
std,91.472183,0.000016,0.002926,0.002916,0.000051,5.666988e-08,1.962257e-07
min,62.000000,0.150901,0.002027,0.002521,0.000038,5.030188e-04,5.015299e-04
25%,268.000000,0.150932,0.008305,0.008778,0.000038,5.031313e-04,5.016040e-04
50%,273.000000,0.150933,0.008460,0.008934,0.000055,5.031400e-04,5.016040e-04
75%,393.000000,0.150941,0.012463,0.012924,0.000056,5.031488e-04,5.016454e-04
max,393.000000,0.150941,0.012463,0.012924,0.000240,5.032061e-04,5.023257e-04


In [23]:
pd.DataFrame(extracts_p1).describe()

,Att_DEGREE_PER,Att_COMBINE_PER,YN_COMBINE_PER,YP_COMBINE_PER,Att_INTER_PER,YN_INTER_PER,YP_INTER_PER,Att_INTRA_PER,YN_INTRA_PER,YP_INTRA_PER,Att_M_COMBINE_PER,Att_M_INTER_PER,Att_M_INTRA_PER
count,800.000000,800.000000,800.000000,800.000000,800.000000,8.000000e+02,8.000000e+02,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,244.067500,0.315947,0.000052,0.000022,0.473693,1.054006e-05,4.421587e-06,0.000453,0.000136,0.000058,0.317367,0.023799,0.904505
std,65.217019,0.000129,0.000009,0.000011,0.000010,1.173684e-08,1.171453e-08,0.000367,0.000028,0.000032,0.000001,0.000001,0.000001
min,47.000000,0.315891,0.000042,0.000010,0.473689,1.053221e-05,4.403193e-06,0.000293,0.000104,0.000022,0.317363,0.023795,0.904501
25%,212.000000,0.315891,0.000042,0.000010,0.473689,1.053237e-05,4.403730e-06,0.000293,0.000105,0.000022,0.317367,0.023798,0.904504
50%,229.000000,0.315926,0.000048,0.000026,0.473692,1.053242e-05,4.429183e-06,0.000394,0.000122,0.000070,0.317367,0.023798,0.904504
75%,308.000000,0.315937,0.000063,0.000036,0.473693,1.055813e-05,4.429183e-06,0.000426,0.000169,0.000098,0.317369,0.023800,0.904506
max,308.000000,0.316450,0.000063,0.000036,0.473731,1.055813e-05,4.429183e-06,0.001889,0.000169,0.000099,0.317369,0.023800,0.904506


In [24]:
pd.DataFrame(extracts_p_t1).describe()

,Att_DEGREE_PER,Att_COMBINE_PER,YN_COMBINE_PER,YP_COMBINE_PER,Att_INTER_PER,YN_INTER_PER,YP_INTER_PER,Att_INTRA_PER,YN_INTRA_PER,YP_INTRA_PER,Att_M_COMBINE_PER,Att_M_INTER_PER,Att_M_INTRA_PER
count,200.000000,200.000000,200.000000,200.000000,200.000000,2.000000e+02,2.000000e+02,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,306.775000,0.450205,0.000075,0.000029,0.900037,1.055813e-05,4.429183e-06,0.000372,0.000140,0.000054,0.497982,0.090546,0.905419
std,91.472183,0.000169,0.000014,0.000016,0.000031,1.698317e-21,8.491585e-22,0.000308,0.000027,0.000031,0.000059,0.000011,0.000107
min,62.000000,0.450126,0.000058,0.000013,0.900023,1.055813e-05,4.429183e-06,0.000230,0.000105,0.000022,0.497877,0.090527,0.905227
25%,268.000000,0.450126,0.000066,0.000013,0.900023,1.055813e-05,4.429183e-06,0.000230,0.000122,0.000022,0.497994,0.090548,0.905441
50%,273.000000,0.450182,0.000071,0.000031,0.900033,1.055813e-05,4.429183e-06,0.000330,0.000132,0.000058,0.498005,0.090550,0.905459
75%,393.000000,0.450185,0.000090,0.000038,0.900033,1.055813e-05,4.429183e-06,0.000337,0.000169,0.000071,0.498015,0.090553,0.905477
max,393.000000,0.450790,0.000090,0.000052,0.900143,1.055813e-05,4.429183e-06,0.001437,0.000169,0.000099,0.498065,0.090558,0.905572
